In [1]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.3 MB/s eta 0:00:

In [2]:
import os
import urllib.request
import json
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.schema import Document
import requests

In [3]:
os.environ['OPENAI_API_KEY'] = '여러분의 키 값'

In [4]:
# 분석할 PDF 파일을 웹에서 다운로드.
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서.pdf"

response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)

print(f"{filename} 다운로드 완료")

2023_북한인권보고서.pdf 다운로드 완료


In [5]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)  # GPT-4o를 언어 모델로 사용
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")  # 임베딩 모델 사용

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 문서를 300자 단위로 분할
    chunk_overlap=100,  # 문맥 유지를 위해 청크 간 100자 중복
)

# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")  # PDF 문서 로더
documents = loader.load()  # 문서에서 텍스트 추출
chunks = text_splitter.split_documents(documents)  # 문서 분할
vector_store = Chroma.from_documents(chunks, embed_model)  # 추출된 텍스트로 벡터 인덱스 생성

In [6]:
class DocumentScorer:
    # LLM을 사용해 문서의 관련성을 정밀하게 평가하고 점수를 매기는 클래스

    def __init__(self, llm):
        self.llm = llm

    def evaluate_document(self, query: str, content: str) -> float:
        # LLM을 사용해 문서와 쿼리 간의 의미적 관련성을 1-10점으로 평가
        prompt = f"""
        아래 주어진 질문과 문서의 관련성을 평가해주세요.

        [평가 기준]
        - 문서가 질문에서 요구하는 정보를 직접적으로 포함하면 8-10점
        - 문서가 질문과 관련된 맥락을 포함하지만 직접적인 답이 아니면 4-7점
        - 문서가 질문과 거의 관련이 없으면 1-3점

        [주의사항]
        - 단순히 비슷한 단어가 등장하는 것은 높은 점수의 근거가 될 수 없습니다
        - 질문의 의도와 문맥을 정확히 파악하여 평가해주세요
        - 시간, 장소, 수치 등 구체적인 정보의 일치 여부를 중요하게 고려해주세요

        질문: {query}
        문서: {content}

        응답은 반드시 다음 JSON 형식이어야 합니다. 백틱은 쓰지마십시오.:
        {{"relevance_score": float}}
        """

        try:
            # LLM에 프롬프트를 전송하고 JSON 형식의 응답을 받음
            response = self.llm.invoke(prompt)
            # 응답에서 relevance_score 값을 추출
            score = json.loads(response.content)["relevance_score"]
            # 점수를 float로 변환하여 반환
            return float(score)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return 5.0  # 에러 발생시 중간 점수로 처리하여 시스템 안정성 유지

    def postprocess_documents(self, documents: List[Document], query: str) -> List[Document]:
        # 벡터 검색으로 찾은 4개 문서를 LLM으로 재평가하여 최적의 2개 선택
        print('\n=== LLM이 4개의 검색 결과에 대해서 관련성을 평가합니다. ===')
        scored_docs = []
        for doc in documents:
            # 현재 처리 중인 문서에서 순수 텍스트 컨텐츠만 추출
            content = doc.page_content
            # LLM으로 문서 관련성 점수 계산 (1-10 사이 점수)
            score = self.evaluate_document(query, content)
            # 디버깅/모니터링을 위해 각 문서의 내용과 점수를 출력
            print(f"\nLLM 기반의 평가:\n{content}\n=> 점수: {score}\n")
            # 현재 문서와 계산된 점수를 튜플로 저장
            scored_docs.append((doc, score))

        # 모든 문서를 점수 기준 내림차순으로 정렬하고 상위 2개만 선택하여 반환
        ranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:2]]

In [7]:
class SemanticRanker:
    # 벡터 검색 결과에 LLM 기반 의미적 평가를 적용하여 최적의 문서를 선별하는 시스템

    def __init__(self, vector_store, scorer):
        # 생성자에서 벡터 검색용 저장소와 LLM 기반 문서 평가기 인스턴스를 받아 저장
        self.vector_store = vector_store  # 벡터 검색용 저장소
        self.scorer = scorer  # LLM 기반 문서 평가기

    def retrieve(self, query: str) -> List[Document]:
        # 벡터 검색으로 유사도 기반 후보 문서 4개를 추출하고 LLM으로 재평가
        vector_results = self.vector_store.similarity_search(query, k=4)

        # 초기 벡터 검색 결과를 디버깅/분석용으로 출력
        print("\n=== 실제 검색 결과 (Top 4) ===")
        for i, doc in enumerate(vector_results, 1):
            print(f"\n검색 문서 {i}:")
            print(doc.page_content)

        # LLM으로 문서들을 재평가하고 재정렬하여 최적의 2개 선택
        reranked_results = self.scorer.postprocess_documents(vector_results, query)

        # 최종 선별된 문서를 디버깅/분석용으로 출력
        print("\n=== LLM의 리랭킹 결과 (Top 2) ===")
        for i, doc in enumerate(reranked_results, 1):
            print(f"\n검색 문서 {i}:")
            print(doc.page_content)

        return reranked_results

In [8]:
# 문서 평가 및 검색 시스템 선언(초기화)
scorer = DocumentScorer(llm)  # LLM 기반 문서 평가기 생성
ranker = SemanticRanker(vector_store, scorer)  # 벡터 검색과 LLM 평가를 결합한 시스템 생성

In [9]:
# 최종 답변 생성 함수
def generate_final_answer(query: str, documents: List[Document]) -> str:
    context = "\n\n".join([doc.page_content for doc in documents])

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
    검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {query}

    답변:"""

    response = llm.invoke(prompt)
    return response.content

# 실제 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
print(f"\n질문: {query}")

# 리랭킹을 통해 최적의 문서 2개 선택
best_documents = ranker.retrieve(query)

# 선택된 문서로 최종 답변 생성
final_answer = generate_final_answer(query, best_documents)
print(f"\n최종 답: {final_answer}")


질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식

=== 실제 검색 결과 (Top 4) ===

검색 문서 1:
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매

검색 문서 2:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았

검색 문서 3:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가

검색 문서 4:
한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정도였다고 하였다. 2019년 양강도의 합영회사는 노동자에게 매달 
9~12만원의 보수를 지급하고, 1년에 한번 쌀 25kg을 지급하였다는 
진술이 있었다. 또한 2020년 합영회사에서는 보수를 성과만큼 받았
다고 하는데, 숙련공은